In [1]:
# Libraries
from pathlib import Path
import os
import sys
import csv
import shutil
import itertools
import random
import PIL
import pytesseract
import numpy as np
import scipy.ndimage as nd
import pandas as pd
from collections import Counter
from PIL import Image, ImageChops, ImageStat
from random import sample
import re
import glob
from datetime import datetime
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

In [36]:
start_time = datetime.now()
current_dir = os.getcwd()
doc_name = os.path.basename(current_dir) 
doc_name
current_dir

'/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie'

In [3]:
sys.path.append("/home/lucas-jerusalimiec/Documents/OCR Text/Notebooks")
from cropfunctions  import rotation_angle, trim, get_bands, simp_bd, combine_bbox
from ocr_func import cutMarg, OCRtestImg, testList, tsvOCR, adjustImg

In [4]:
# Display a list of languages in their 3-letter codes supported by Tesseract.
print(pytesseract.get_languages(config=''))

['eng', 'fra', 'frm', 'lat', 'osd']


In [5]:
# Create Directories
output_folder = Path('./output')
output_folder.mkdir(exist_ok=True)
outDir = "./output"
jpgs_folder = Path.cwd()

In [9]:
limit = len(glob.glob('**/*.jpg', recursive=True))
print('file n = ', limit)

file n =  450


In [11]:
if limit >= 50:
    sample_n = 10
elif limit >= 20:
    sample_n = 5
elif limit >= 10:
    sample_n = 3
elif limit >= 5:
    sample_n = 2
else:
    sample_n = 1
    
print('sample n = ', sample_n)

sample n =  10


In [12]:
correct_sides = ['left', 'right']
while True:
    side_input = input('Please enter a starting side: ').lower()
    if side_input in correct_sides:
        break
    else:
        print('Incorrect value. Please try again.')

Please enter a starting side:  right


In [13]:
if side_input == 'right':
    side_1 = 'right'
    side_2 = 'left'
elif side_input == 'left':
    side_1 = 'left'
    side_2 = 'right'

side = [side_1 if i % 2 == 0 else side_2 for i in range(limit)]
print(side[0:4])

['right', 'left', 'right', 'left']


In [16]:
# List of metadata dictionaries
meta = []
count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
        row_dict = dict()
        row_dict["filename"] = os.path.relpath(jpg)
        row_dict["side"] = side[count]
        row_dict["start_section"] = False
        row_dict["find_top"] = False
        #row_dict["find_top"] = jpg.name.endswith('0000.jpg')
        row_dict["background"] = 0
        row_dict["bbox"] = []
        meta.append(row_dict)
        count = count+1
            
print(meta[0])

{'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg', 'side': 'right', 'start_section': False, 'find_top': False, 'background': 0, 'bbox': []}


In [17]:
# Extract find_top
find_top = [d['find_top'] for d in meta]
print(find_top[0:3])

[False, False, False]


In [19]:
#Combined rotation, greyscale (inverse), and trim workflow metadata
### Convert multiple jpegs into a set of greyscale images ###
buff=10
background = []
bbox = []
fname = []
diff_open = []
angle = []
count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    print(f'Processing {os.path.relpath(jpg)}')
    img = Image.open(jpg)
    best_angle = rotation_angle(img)
    diff0, background0, bbox0 = (trim(img, angle=best_angle, buff=buff, find_top = find_top[count]))
    bbox.append(bbox0)
    background.append(background0)
    fname.append(os.path.relpath(jpg))
    diff_open.append(diff0)
    angle.append(best_angle)
    count = count + 1

print('Images successfully processed')

Processing Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0011.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0000.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0001.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0002.jpg
Processing Démonomanie I.

In [20]:
# Update meta with diffs
diff_list = []
count = 0

for file in fname:
        diff_dict = dict()
        diff_dict["filename"] = fname[count]
        diff_dict["bbox"] = bbox[count]
        diff_dict["background"] = background[count]
        diff_dict["angle"] = angle[count]
        diff_list.append(diff_dict)
        count = count+1
            
for original_dict in meta:
    for update_dict in diff_list:
        if original_dict["filename"] == update_dict["filename"]:
            original_dict.update(update_dict)     
meta

[{'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (221, 207, 177),
  'bbox': (0, 0, 3556, 5456),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 203, 173),
  'bbox': (9, 0, 3556, 5226),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (222, 207, 178),
  'bbox': (0, 0, 3556, 5355),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0003.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 204, 174),
  'bbox': (0, 0, 3556, 5275),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0004.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': Fals

In [23]:
#Bands creation workflow
bheight = 50
band_dict = []

count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    print(f'Processing {os.path.relpath(jpg)}')
    band0 =  get_bands(diff_open[count], bheight=bheight)
    band_dict.append(band0)
    
    count = count + 1

print('Bands successfully created')

Processing Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0011.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0000.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0001.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0002.jpg
Processing Démonomanie I.

In [24]:
#Bands application workflow
count = 0
cut_list = []
pad = 10
#allow = (0.1, 0.3) #parameters for Bodin
allow = (0.1, 0.1) #parameters for L'Hospital, Démonomanie, Théatre
freq = 0.8
minfreq = 0.1

for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    print(f'Processing {os.path.relpath(jpg)}')
    cut0 = simp_bd(band_dict=band_dict[count], diff=diff_open[count], side=side[count], width=diff_open[count].size[0],
                   pad=pad, allow = allow, freq = freq, minfreq = minfreq)
    cut_list.append(cut0)
    count = count + 1

print('Bands successfully applied')
cut_list[0:3]

Processing Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1/images/Démonomanie I.1_image_0011.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0000.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0001.jpg
Processing Démonomanie I.2/images/Démonomanie I.2_image_0002.jpg
Processing Démonomanie I.

[3556, 0, 3556]

In [25]:
# Update meta with cuts
count = 0
cut_update=[]
for file in fname:
        cut_dict = dict()
        cut_dict["filename"] = fname[count]
        cut_dict["cut"] = cut_list[count]
        cut_update.append(cut_dict)
        count = count+1
            
for original_dict in meta:
    for update_dict in cut_update:
        if original_dict["filename"] == update_dict["filename"]:
            original_dict.update(update_dict)     
meta[0:3]

[{'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (221, 207, 177),
  'bbox': (0, 0, 3556, 5456),
  'angle': 0.0,
  'cut': 3556},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 203, 173),
  'bbox': (9, 0, 3556, 5226),
  'angle': 0.0,
  'cut': 0},
 {'filename': 'Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (222, 207, 178),
  'bbox': (0, 0, 3556, 5355),
  'angle': 0.0,
  'cut': 3556}]

In [26]:
# create a bbox for the entire diff image
out_bbox = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    out_bbox0 = [0, 0] + list(diff_open[count].size)
    # Set the side value so that the function 
    # knows which bbox dimension to "cut"
    side_dict = {"left":0, "right":2}
    # Set the outer edge of the page equal to cut, 
    # rather than the original outer edge from diff
    # This creates a bbox that is cropped to include 
    # only the main text areas of diff
    side_0 = side[count]
    out_bbox0[side_dict[side_0]] = cut_list[count]
    out_bbox.append(out_bbox0)
    count = count + 1

print('Bbox margins successfully defined')
out_bbox[0:3]

Bbox margins successfully defined


[[0, 0, 3556, 5456], [0, 0, 3547, 5226], [0, 0, 3556, 5355]]

In [27]:
#Creating cropped image boundaries
total_bbox = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    total_bbox0 = combine_bbox(bbox[count], out_bbox[count])
    total_bbox0=list(total_bbox0)
    total_bbox.append(total_bbox0)
    count = count + 1

print('Bbox dimensions successfully adjusted')
total_bbox[0:3]

Bbox dimensions successfully adjusted


[[0, 0, 3556, 5456], [9, 0, 3556, 5226], [0, 0, 3556, 5355]]

In [28]:
# Extract angle
angle_list = [d['angle'] for d in meta]
print(angle_list[0:3])

[0.0, 0.0, 0.0]


In [29]:
#metadata collected
filename_list = [d['filename'] for d in meta]
side_list = [d['side'] for d in meta]
meta_final = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
        meta_list = [filename_list[count], angle_list[count], side_list[count], cut_list[count]]
        meta_list.extend(background[count])
        meta_list.extend(total_bbox[count])
        meta_final.append(meta_list)
        count = count+1
 
meta_final[0:3]

[['Démonomanie I.1/images/Démonomanie I.1_image_0000.jpg',
  0.0,
  'right',
  3556,
  221,
  207,
  177,
  0,
  0,
  3556,
  5456],
 ['Démonomanie I.1/images/Démonomanie I.1_image_0001.jpg',
  0.0,
  'left',
  0,
  220,
  203,
  173,
  9,
  0,
  3556,
  5226],
 ['Démonomanie I.1/images/Démonomanie I.1_image_0002.jpg',
  0.0,
  'right',
  3556,
  222,
  207,
  178,
  0,
  0,
  3556,
  5355]]

In [30]:
#metadata csv
headers = ["filename","angle","side","cut","backR","backG","backB",
           "bbox1","bbox2","bbox3","bbox4"]
with open(f'{outDir}/{doc_name}_metadata.csv',"w",newline="") as outfile:
    writer=csv.writer(outfile)
    writer.writerow(headers)
    for row in meta_final:
        writer.writerow(row)

In [31]:
df = pd.read_csv(f'{outDir}/{doc_name}_metadata.csv')
df.head()

,filename,angle,side,cut,backR,backG,backB,bbox1,bbox2,bbox3,bbox4
0,Démonomanie I.1/images/Démonomanie I.1_image_0...,0.0,right,3556,221,207,177,0,0,3556,5456
1,Démonomanie I.1/images/Démonomanie I.1_image_0...,0.0,left,0,220,203,173,9,0,3556,5226
2,Démonomanie I.1/images/Démonomanie I.1_image_0...,0.0,right,3556,222,207,178,0,0,3556,5355
3,Démonomanie I.1/images/Démonomanie I.1_image_0...,0.0,left,0,220,204,174,0,0,3556,5275
4,Démonomanie I.1/images/Démonomanie I.1_image_0...,-0.5,right,3556,222,208,178,0,0,3556,5306


In [32]:
# Initialize variables
metalist = f"{outDir}/{doc_name}_metadata.csv"
csv1 = pd.read_csv(metalist)

In [34]:
pool = []
csvf = csv1.set_index("filename")
#csvf = csv1[csv1["filename"].str.startswith(doc_name)].set_index("filename")
csvf['file'] = csvf.index
for row in csvf.itertuples():
    pool.append(os.path.normpath(os.path.join(current_dir, row.file)))

pool = sample(pool, sample_n)

pool

['/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie II.6/images/Démonomanie II.6_image_0001.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie IV.5/images/Démonomanie IV.5_image_0013.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie II.6/images/Démonomanie II.6_image_0012.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie IV.1/images/Démonomanie IV.1_image_0008.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie IV.2/images/Démonomanie IV.2_image_0011.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie IV.2/images/Démonomanie IV.2_image_0013.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/Démonomanie/Démonomanie I.5/images/Démonomanie I.5_image_0004.jpg',
 '/home/lucas-jerusalimiec/Documents/OCR Bin/Split/Bodin Repair/

In [ ]:
#Get images for files in sample, cut margins and make a test list
imgs = []
results = []

for img in pool:

    #get image name
    name = os.path.relpath(img, start=current_dir)
    print('Testing', name)
    #get values for cutting margins
    rotate = csvf.loc[name]["angle"]
    left = csvf.loc[name]["bbox1"]
    up = csvf.loc[name]["bbox2"]
    right = csvf.loc[name]["bbox3"]
    lower = csvf.loc[name]["bbox4"]
    bkgcol = (csvf.loc[name]["backR"], csvf.loc[name]["backG"], csvf.loc[name]["backB"])

    #cut the margins
    img = cutMarg(img = img, rotate = rotate, left = left, up = up, right = right,
                 lower = lower, border = 0, bkgcol = bkgcol)


    #add the new image to the list
    imgs.append(img)

    #perform an OCR test on the new image and add the results to the list
    results.append(OCRtestImg(img))

Testing Démonomanie II.6/images/Démonomanie II.6_image_0001.jpg
Testing Démonomanie IV.5/images/Démonomanie IV.5_image_0013.jpg
Testing Démonomanie II.6/images/Démonomanie II.6_image_0012.jpg
Testing Démonomanie IV.1/images/Démonomanie IV.1_image_0008.jpg


In [26]:
#create a testList object with the  images and results
testSample = testList(imgs, results)

In [27]:
#list of available tests
print('conTests = ["color", "brightness", "contrast", "autocontrast", "sharpness"]')
print('boolTests = ["invert", "blur", "sharpen", "smooth", "xsmooth"]')

conTests = ["color", "brightness", "contrast", "autocontrast", "sharpness"]
boolTests = ["invert", "blur", "sharpen", "smooth", "xsmooth"]


In [28]:
#set up a dict of recommended adjustments and perform tests
adjustments = { "volume": doc_name, "color": 1.0, "invert": False, 
                "autocontrast": 0, "blur": False, "sharpen": False, 
                "smooth": False, "xsmooth": False }

In [29]:
#color test
print("color test")
testRes = testSample.adjustTest("color", levels = [1,.75,.5,.25,0])

color test



                                          color1  color0.75  color0.5  \
Harangue - lit de justice_image_0003.jpg      35         35        35   
Harangue - lit de justice_image_0001.jpg      37         37        37   
Harangue - lit de justice_image_0002.jpg      28         28        28   

                                          color0.25  color0  totaltok  
Harangue - lit de justice_image_0003.jpg         35      35       219  
Harangue - lit de justice_image_0001.jpg         37      37       223  
Harangue - lit de justice_image_0002.jpg         28      28       229  

TOTAL READABILITY
color1: 85.097
color0.75: 85.097
color0.5: 85.097
color0.25: 85.097
color0: 85.097


In [30]:
best = float(testRes["best_adjustment"].replace("color", ""))
if best != 1.0:
    testSample = testSample.adjustImg(color = best)
    adjustments["color"] = best
    

In [31]:
#autocontrast test
print("autocontrast test")
testRes = testSample.adjustTest("autocontrast", levels = [0,2,4,6,8])
best = float(testRes["best_adjustment"].replace("autocontrast", ""))
if best != 0.0:
    testSample = testSample.adjustImg(autocontrast = best)
    adjustments["autocontrast"] = best

autocontrast test



                                          autocontrast0  autocontrast2  \
Harangue - lit de justice_image_0003.jpg             35             35   
Harangue - lit de justice_image_0001.jpg             37             37   
Harangue - lit de justice_image_0002.jpg             28             28   

                                          autocontrast4  autocontrast6  \
Harangue - lit de justice_image_0003.jpg             35             34   
Harangue - lit de justice_image_0001.jpg             37             37   
Harangue - lit de justice_image_0002.jpg             28             29   

                                          autocontrast8  totaltok  
Harangue - lit de justice_image_0003.jpg             36       219  
Harangue - lit de justice_image_0001.jpg             37       223  
Harangue - lit de justice_image_0002.jpg             29       229  

TOTAL READABILITY
autocontrast0: 85.097
autocontrast2: 85.097
autocontrast4: 85.097
autocontrast6: 85.097
autoco

In [32]:
#blur test
print("blur test")
testRes = testSample.adjustTest("blur")
if testRes["best_adjustment"] == "blurTrue":
    testSample = testSample.adjustImg(blur = True)
    adjustments["blur"] = True

blur test



                                          blurTrue  blurFalse  totaltok
Harangue - lit de justice_image_0003.jpg        36         35       219
Harangue - lit de justice_image_0001.jpg        37         37       223
Harangue - lit de justice_image_0002.jpg        27         28       229

TOTAL READABILITY
blurTrue: 85.097
blurFalse: 85.097


In [33]:
#sharpen test
print("sharpen test")
testRes = testSample.adjustTest("sharpen")
if testRes["best_adjustment"] == "sharpenTrue":
    testSample = testSample.adjustImg(sharpen = True)
    adjustments["sharpen"] = True

sharpen test



                                          sharpenTrue  sharpenFalse  totaltok
Harangue - lit de justice_image_0003.jpg           35            36       218
Harangue - lit de justice_image_0001.jpg           37            37       223
Harangue - lit de justice_image_0002.jpg           27            27       230

TOTAL READABILITY
sharpenTrue: 85.246
sharpenFalse: 85.097


In [34]:
#smooth test
print("smooth test")
testRes = testSample.adjustTest("smooth")
if testRes["best_adjustment"] == "smoothTrue":
    testSample = testSample.adjustImg(smooth = True)
    adjustments["smooth"] = True

    #xsmooth test
    print("xsmooth test")
    testRes = testSample.adjustTest("xsmooth")
    if testRes["best_adjustment"] == "xsmoothTrue":
        testSample = testSample.adjustImg(xsmooth = True)
        adjustments["xsmooth"] = True

smooth test



                                          smoothTrue  smoothFalse  totaltok
Harangue - lit de justice_image_0003.jpg          37           35       219
Harangue - lit de justice_image_0001.jpg          37           37       223
Harangue - lit de justice_image_0002.jpg          27           27       230

TOTAL READABILITY
smoothTrue: 84.97
smoothFalse: 85.268


In [35]:
#post-test adjustment levels
adjustments
adjustments_list = [*adjustments.values()]
adjustments_list

['Harangue - lit de justice', 1.0, False, 0, True, True, False, False]

In [36]:
#adjustments csv
headers = ["volume","color","invert","autocontrast","blur","sharpen","smooth",
           "xsmooth"]
with open(f"{outDir}/{doc_name}_adjustments.csv","w",newline="") as outfile:
    writer=csv.writer(outfile)
    writer.writerow(headers)
    writer.writerow(adjustments_list)

In [37]:
#Set paths for metadata files
margdata = f"{outDir}/{doc_name}_metadata.csv"
adjdata = f"{outDir}/{doc_name}_adjustments.csv"

#Read metadata files into memory
#Read csvs
margcsv = pd.read_csv(margdata)
adjcsv = pd.read_csv(adjdata)

In [38]:
#merge csvs
margcsv['key'] = 0
adjcsv['key'] = 0

# Perform the Cartesian product
fulldata = pd.merge(margcsv, adjcsv, on='key')

# Drop the temporary key column
fulldata.drop('key', axis=1, inplace=True)

# Save the combined dataframe to a new CSV file
fulldata.to_csv(f'{outDir}/{doc_name}_fulldata.csv', index=False)

In [39]:
#create OCR files
fulldata_path = f"{outDir}/{doc_name}_fulldata.csv"
fulldata = pd.read_csv(fulldata_path)
#Loop through section
for row in fulldata.itertuples():

    img = os.path.normpath(os.path.join(current_dir, row.filename))
    print(datetime.now().strftime("%H:%M") + " Processing " + row.filename + "...")

    #set up margin cutting
    cuts = {"rotate" : row.angle,
            "left" : row.bbox1,
            "up" : row.bbox2,
            "right" : row.bbox3,
            "lower" : row.bbox4,
            "border" : 0,
            "bkgcol" : (row.backR, row.backG, row.backB)}

    #set up image adjustment            
    adjustments = {"color": row.color, 
                   "autocontrast": row.autocontrast,
                   "blur": row.blur,
                   "sharpen": row.sharpen,
                   "smooth": row.smooth,
                   "xsmooth": row.xsmooth}

    #OCR the image
    tsvOCR((adjustImg(cutMarg(img, **cuts), **adjustments)), 
           savpath = os.path.normpath(os.path.join(outDir, doc_name + ".txt")),
           tsvfile = doc_name + "_data.tsv")

print('Done OCR job')

21:53 Processing Harangue - lit de justice_image_0000.jpg...
21:53 Processing Harangue - lit de justice_image_0001.jpg...
21:53 Processing Harangue - lit de justice_image_0002.jpg...
21:53 Processing Harangue - lit de justice_image_0003.jpg...
21:53 Processing Harangue - lit de justice_image_0004.jpg...
21:53 Processing Harangue - lit de justice_image_0005.jpg...
21:53 Processing Harangue - lit de justice_image_0006.jpg...
21:53 Processing Harangue - lit de justice_image_0007.jpg...
21:53 Processing Harangue - lit de justice_image_0008.jpg...
21:53 Processing Harangue - lit de justice_image_0009.jpg...
21:53 Processing Harangue - lit de justice_image_0010.jpg...
21:53 Processing Harangue - lit de justice_image_0011.jpg...
21:53 Processing Harangue - lit de justice_image_0012.jpg...
21:53 Processing Harangue - lit de justice_image_0013.jpg...
21:53 Processing Harangue - lit de justice_image_0014.jpg...
21:53 Processing Harangue - lit de justice_image_0015.jpg...
Done OCR job


In [40]:
end_time = datetime.now()
time_consumed = end_time - start_time
avg_time = time_consumed / limit
print(f'Time consumed by {doc_name} notebook run: {time_consumed}')
print(f'{limit} pages')
print(f'{avg_time} per page')

Time consumed by Harangue - lit de justice notebook run: 0:03:47.597763
16 pages
0:00:14.224860 per page
